In [1]:
%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pydicom
import os
import numpy as np
from Classifier import MGClassifier
import utils
import torch
import time

## row min/max , col min/max
## 2294 4096 1914 3328


Using TensorFlow backend.


In [2]:
## read files and their infos
# searching dcm files 
pt_id_lossy = {}
path_data = '/share_folder/data/breast_hu/lossy/'
for dir_name,_,files in os.walk(path_data):
    try:  
        dir_name.split('/')[6]
    except: pass
    else:
        pt_str = dir_name.split('/')[6]
        #pt_status = dir_name.split('/')[5]
        pt_id_lossy[pt_str] = dir_name


pt_id_lossless = {}
path_data = '/share_folder/data/breast_hu/lossless/'
for dir_name,_,files in os.walk(path_data):
    try:  
        dir_name.split('/')[6]
    except: pass
    else:
        pt_str = dir_name.split('/')[6]
       # pt_status = dir_name.split('/')[5]
        pt_id_lossless[pt_str] = dir_name

## check files 
if set(pt_id_lossless.keys()).difference(set( pt_id_lossy.keys())) == set():
    print('Complementary')

pt_id = list(pt_id_lossless.keys())
    

Complementary


In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(pt_id,test_size=0.25, random_state=42)
batch_size =8

In [4]:
model_lossy = utils.DeepModel(MGClassifier(), num_classes =3, criterion = torch.nn.CrossEntropyLoss() ,note='lossy')
model_lossless = utils.DeepModel(MGClassifier(), num_classes =3, criterion = torch.nn.CrossEntropyLoss() ,note='lossless')


Model name: MGClassifierlossy10020742_9393
Model Info: MGClassifier(
  (encoder): Sequential(
    (0): Conv2d(1, 4, kernel_size=(15, 15), stride=(3, 3), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(4, 8, kernel_size=(15, 15), stride=(2, 2), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(8, 16, kernel_size=(7, 7), stride=(2, 2), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (net): ResNet(
    (conv1): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False

In [5]:
for i in range(0,len(train[:8]),batch_size):
    batch_ids = train[i:i+batch_size]
    print('num_batch = ',i)
    X_batch = []
    y_batch = []
    t = time.process_time() 
    for j in range (batch_size):
        pt_mammo = []
        path = pt_id_lossy[batch_ids[j]]
        if path.split('/')[5] =='normal':
            y_batch.append(0)
        elif path.split('/')[5] =='benign':
            y_batch.append(1)
        else:
            y_batch.append(2)
        tmp = np.load(path+'/RMLO.npy')#pydicom.read_file(path+'/RMLO.dcm').pixel_array
        tmp = np.pad(tmp,((0, 4096-tmp.shape[0]), (0, 3328-tmp.shape[1])),mode='constant')
        tmp = (tmp-tmp.min()) / (tmp.max()-tmp.min())
        tmp = tmp.reshape(1,tmp.shape[0],tmp.shape[1])
        pt_mammo.append(tmp )
        
        tmp = np.load(path+'/LMLO.npy') #pydicom.read_file(path+'/LMLO.dcm').pixel_array
        tmp = np.pad(tmp,((0, 4096-tmp.shape[0]), (0, 3328-tmp.shape[1])),mode='constant')
        tmp = (tmp-tmp.min()) / (tmp.max()-tmp.min())
        tmp = tmp.reshape(1,tmp.shape[0],tmp.shape[1])
        pt_mammo.append(tmp)
        
        tmp = np.load(path+'/RCC.npy') #pydicom.read_file(path+'/RCC.dcm').pixel_array
        tmp = np.pad(tmp,((0, 4096-tmp.shape[0]), (0, 3328-tmp.shape[1])),mode='constant')
        tmp = (tmp-tmp.min()) / (tmp.max()-tmp.min())
        tmp = tmp.reshape(1,tmp.shape[0],tmp.shape[1])
        pt_mammo.append(tmp)
        
        tmp = np.load(path+'/LCC.npy') #pydicom.read_file(path+'/LCC.dcm').pixel_array
        tmp = np.pad(tmp,((0, 4096-tmp.shape[0]), (0, 3328-tmp.shape[1])),mode='constant')
        tmp = (tmp-tmp.min()) / (tmp.max()-tmp.min())
        tmp = tmp.reshape(1,tmp.shape[0],tmp.shape[1])
        pt_mammo.append(tmp) 
        
        X_batch.append(np.array(pt_mammo))
    print('read time:', '{:.3f}'.format(time.process_time() - t),'sec')
    X_batch = np.array(X_batch)    
    y_batch = np.array(y_batch)  
    model_lossy.training(X_batch,y_batch,batch_size = batch_size,epoch=1)

num_batch =  0
read time: 102.202 sec
Training data shape: (8, 4, 1, 4096, 3328) (8,)
~~~~~~~~~~~~~~~~~~~~ Training ~~~~~~~~~~~~~~~~~~~~
2 GPUs used!
0th loss: 1.0807 | Precision: 0.0000 | Recall: 0.0000
0th loss:1.08072
*** Complete! Training time: 4.940 sec ***


/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [6]:
for i in range(0,len(train[:8]),batch_size):
    batch_ids = train[i:i+batch_size]
    print('num_batch = ',i)
    X_batch = []
    y_batch = []
    t = time.process_time() 
    for j in range (batch_size):
        pt_mammo = []
        path = pt_id_lossless[batch_ids[j]]
        if path.split('/')[5] =='normal':
            y_batch.append(0)
        elif path.split('/')[5] =='benign':
            y_batch.append(1)
        else:
            y_batch.append(2)
        tmp = np.load(path+'/RMLO.npy')#pydicom.read_file(path+'/RMLO.dcm').pixel_array
        tmp = np.pad(tmp,((0, 4096-tmp.shape[0]), (0, 3328-tmp.shape[1])),mode='constant')
        tmp = (tmp-tmp.min()) / (tmp.max()-tmp.min())
        tmp = tmp.reshape(1,tmp.shape[0],tmp.shape[1])
        pt_mammo.append(tmp )
        
        tmp = np.load(path+'/LMLO.npy') #pydicom.read_file(path+'/LMLO.dcm').pixel_array
        tmp = np.pad(tmp,((0, 4096-tmp.shape[0]), (0, 3328-tmp.shape[1])),mode='constant')
        tmp = (tmp-tmp.min()) / (tmp.max()-tmp.min())
        tmp = tmp.reshape(1,tmp.shape[0],tmp.shape[1])
        pt_mammo.append(tmp)
        
        tmp = np.load(path+'/RCC.npy') #pydicom.read_file(path+'/RCC.dcm').pixel_array
        tmp = np.pad(tmp,((0, 4096-tmp.shape[0]), (0, 3328-tmp.shape[1])),mode='constant')
        tmp = (tmp-tmp.min()) / (tmp.max()-tmp.min())
        tmp = tmp.reshape(1,tmp.shape[0],tmp.shape[1])
        pt_mammo.append(tmp)
        
        tmp = np.load(path+'/LCC.npy') #pydicom.read_file(path+'/LCC.dcm').pixel_array
        tmp = np.pad(tmp,((0, 4096-tmp.shape[0]), (0, 3328-tmp.shape[1])),mode='constant')
        tmp = (tmp-tmp.min()) / (tmp.max()-tmp.min())
        tmp = tmp.reshape(1,tmp.shape[0],tmp.shape[1])
        pt_mammo.append(tmp) 
        
        X_batch.append(np.array(pt_mammo))
    print('read time:', '{:.3f}'.format(time.process_time() - t),'sec')
    X_batch = np.array(X_batch)    
    y_batch = np.array(y_batch)  
    model_lossy.training(X_batch,y_batch,batch_size = batch_size,epoch=1)

num_batch =  0
read time: 77.956 sec
Training data shape: (8, 4, 1, 4096, 3328) (8,)
~~~~~~~~~~~~~~~~~~~~ Training ~~~~~~~~~~~~~~~~~~~~
2 GPUs used!


RuntimeError: Caught RuntimeError in replica 1 on device 1.
Original Traceback (most recent call last):
  File "/root/anaconda3/lib/python3.6/site-packages/torch/nn/parallel/parallel_apply.py", line 60, in _worker
    output = module(*input, **kwargs)
  File "/root/anaconda3/lib/python3.6/site-packages/torch/nn/modules/module.py", line 547, in __call__
    result = self.forward(*input, **kwargs)
  File "/root/anaconda3/lib/python3.6/site-packages/torch/nn/parallel/data_parallel.py", line 146, in forward
    "them on device: {}".format(self.src_device_obj, t.device))
RuntimeError: module must have its parameters and buffers on device cuda:0 (device_ids[0]) but found one of them on device: cuda:1


In [ ]:
y_batch = []
for i in range(0,len(test[:32]),batch_size):
    batch_ids = test[i:i+batch_size]
    print('num_batch = ',i)
    X_batch = []
    for j in range (batch_size):
        pt_mammo = []
        path = pt_id_lossless[batch_ids[j]]
        if path.split('/')[5] =='normal':
            y_batch.append(0)
        elif path.split('/')[5] =='benign':
            y_batch.append(1)
        else:
            y_batch.append(2)
        tmp = np.load(path+'/RMLO.npy')#pydicom.read_file(path+'/RMLO.dcm').pixel_array
        tmp = np.pad(tmp,((0, 4096-tmp.shape[0]), (0, 3328-tmp.shape[1])),mode='constant')
        tmp = (tmp-tmp.min()) / (tmp.max()-tmp.min())
        tmp = tmp.reshape(1,tmp.shape[0],tmp.shape[1])
        pt_mammo.append(tmp )
        
        tmp = np.load(path+'/LMLO.npy') #pydicom.read_file(path+'/LMLO.dcm').pixel_array
        tmp = np.pad(tmp,((0, 4096-tmp.shape[0]), (0, 3328-tmp.shape[1])),mode='constant')
        tmp = (tmp-tmp.min()) / (tmp.max()-tmp.min())
        tmp = tmp.reshape(1,tmp.shape[0],tmp.shape[1])
        pt_mammo.append(tmp)
        
        tmp = np.load(path+'/RCC.npy') #pydicom.read_file(path+'/RCC.dcm').pixel_array
        tmp = np.pad(tmp,((0, 4096-tmp.shape[0]), (0, 3328-tmp.shape[1])),mode='constant')
        tmp = (tmp-tmp.min()) / (tmp.max()-tmp.min())
        tmp = tmp.reshape(1,tmp.shape[0],tmp.shape[1])
        pt_mammo.append(tmp)
        
        tmp = np.load(path+'/LCC.npy') #pydicom.read_file(path+'/LCC.dcm').pixel_array
        tmp = np.pad(tmp,((0, 4096-tmp.shape[0]), (0, 3328-tmp.shape[1])),mode='constant')
        tmp = (tmp-tmp.min()) / (tmp.max()-tmp.min())
        tmp = tmp.reshape(1,tmp.shape[0],tmp.shape[1])
        pt_mammo.append(tmp) 
        
        X_batch.append(np.array(pt_mammo))
    X_batch = np.array(X_batch)    
    model.test(X_batch,batch_size = batch_size)
y_batch = np.array(y_batch)  
model_lossless.evaluation(y_batch)

In [ ]:
y_batch = []
for i in range(0,len(test[:32]),batch_size):
    batch_ids = test[i:i+batch_size]
    print('num_batch = ',i)
    X_batch = []
    for j in range (batch_size):
        pt_mammo = []
        path = pt_id_lossy[batch_ids[j]]
        if path.split('/')[5] =='normal':
            y_batch.append(0)
        elif path.split('/')[5] =='benign':
            y_batch.append(1)
        else:
            y_batch.append(2)
        tmp = np.load(path+'/RMLO.npy')#pydicom.read_file(path+'/RMLO.dcm').pixel_array
        tmp = np.pad(tmp,((0, 4096-tmp.shape[0]), (0, 3328-tmp.shape[1])),mode='constant')
        tmp = (tmp-tmp.min()) / (tmp.max()-tmp.min())
        tmp = tmp.reshape(1,tmp.shape[0],tmp.shape[1])
        pt_mammo.append(tmp )
        
        tmp = np.load(path+'/LMLO.npy') #pydicom.read_file(path+'/LMLO.dcm').pixel_array
        tmp = np.pad(tmp,((0, 4096-tmp.shape[0]), (0, 3328-tmp.shape[1])),mode='constant')
        tmp = (tmp-tmp.min()) / (tmp.max()-tmp.min())
        tmp = tmp.reshape(1,tmp.shape[0],tmp.shape[1])
        pt_mammo.append(tmp)
        
        tmp = np.load(path+'/RCC.npy') #pydicom.read_file(path+'/RCC.dcm').pixel_array
        tmp = np.pad(tmp,((0, 4096-tmp.shape[0]), (0, 3328-tmp.shape[1])),mode='constant')
        tmp = (tmp-tmp.min()) / (tmp.max()-tmp.min())
        tmp = tmp.reshape(1,tmp.shape[0],tmp.shape[1])
        pt_mammo.append(tmp)
        
        tmp = np.load(path+'/LCC.npy') #pydicom.read_file(path+'/LCC.dcm').pixel_array
        tmp = np.pad(tmp,((0, 4096-tmp.shape[0]), (0, 3328-tmp.shape[1])),mode='constant')
        tmp = (tmp-tmp.min()) / (tmp.max()-tmp.min())
        tmp = tmp.reshape(1,tmp.shape[0],tmp.shape[1])
        pt_mammo.append(tmp) 
        
        X_batch.append(np.array(pt_mammo))
    X_batch = np.array(X_batch)    
    model.test(X_batch,batch_size = batch_size)
y_batch = np.array(y_batch)  
model_lossy.evaluation(y_batch)

In [ ]:
utils.performance_comparison([model])